In [45]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed

# define function to prepare input/output sequences
def prepare_data(data, n_steps_in, n_steps_out, variables):
    X, y = [], []
    for i in range(len(data)-n_steps_in-n_steps_out+1):
        X.append(data[i:i+n_steps_in, variables])
        y.append(data[i+n_steps_in:i+n_steps_in+n_steps_out, variables])
    return np.array(X), np.array(y)

# load dataset
data = pd.read_csv('pollution3.csv', header=0, index_col=0)

# define variables to forecast
forecast_variables = ['pollution', 'temp']

# select subset of variables for model input
model_variables = list(set(data.columns) - set(forecast_variables))

# split into train and test sets
train_size = int(len(data) * 0.8)
train, test = data[0:train_size], data[train_size:len(data)]

# scale data
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)

# define model
n_steps_in, n_steps_out = 7, 3
n_features = len(model_variables)
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_steps_in, n_features)))
model.add(RepeatVector(n_steps_out))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(len(forecast_variables))))

# compile model
model.compile(optimizer='adam', loss='mse')

# define input/output sequences
train_X, train_y = prepare_data(train_scaled, n_steps_in, n_steps_out, model_variables)
test_X, test_y = prepare_data(test_scaled, n_steps_in, n_steps_out, model_variables)

# fit model
history = model.fit(train_X, train_y, epochs=30, batch_size=32, validation_data=(test_X, test_y), verbose=2, shuffle=False)

# make forecast
forecast_X = test_X[:, :, [data.columns.get_loc(variable) for variable in forecast_variables]]
forecast_X_scaled = scaler.transform(forecast_X.reshape(-1, len(forecast_variables)))
forecast_X_scaled = forecast_X_scaled.reshape(-1, n_steps_in, len(forecast_variables))
forecast = model.predict(forecast_X_scaled)

# rescale forecasted data
forecast = scaler.inverse_transform(forecast.reshape(-1, len(forecast_variables)))
forecast = forecast.reshape(-1, n_steps_out, len(forecast_variables))

# evaluate forecast
mse = np.mean((test_y - forecast)**2)
print('MSE: %.3f' % mse)


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [43]:
def prepare_data(data, n_steps_in, n_steps_out, variables):
    X, y = [], []
    for i in range(len(data)-n_steps_in-n_steps_out+1):
        X.append(data[i:i+n_steps_in, variables])
        y.append(data[i+n_steps_in:i+n_steps_in+n_steps_out, variables])
    return np.array(X), np.array(y)
data = pd.read_csv('trig.csv', header=0)

# define variables to forecast
forecast_variables = ['sine', 'cosine']

# select subset of variables for model input
model_variables = list(set(data.columns) - set(forecast_variables))

# split into train and test sets
train_size = int(len(data) * 0.8)
train, test = data[0:train_size], data[train_size:len(data)]

# scale data
scaler = MinMaxScaler()
train_scaled = scaler.fit_transform(train)
test_scaled = scaler.transform(test)


# define input/output sequences
train_X, train_y = prepare_data(train_scaled, n_steps_in, n_steps_out, model_variables)
test_X, test_y = prepare_data(test_scaled, n_steps_in, n_steps_out, model_variables)



IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices